In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle
import shelve
import klepto
from gensim import corpora, models
import feather
from collections import defaultdict
import itertools

In [2]:
poor_data = pd.read_pickle("/Users/Hun/hse_thesis/hse_thesis1.pickle")

In [3]:
add_data = pd.read_csv("/Users/Hun/hse_thesis/metadata.csv", encoding="CP1251",
                        error_bad_lines=False, warn_bad_lines=False,
                       header=None)

/Users/Hun/Python_env/Python3_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
add_data.drop(0, axis=1, inplace=True)

In [5]:
col_names=["author_id", "wall_id", "post_comment_id", "date", "name", "surname", "sex", "city", "region",
      "country", "posts", "comments", "likes", "reposts", "res_postlikes", "res_comments", "university",
       "freinds", "groups", "followers"]

In [6]:
len(col_names)

20

In [7]:
add_data.columns = col_names

In [8]:
add_data["wall_id"] = add_data["wall_id"].apply(lambda text: int(text.split()[-1]))

add_data["post_comment_id_int"] = add_data["post_comment_id"].apply(lambda text: int(text.split()[-1]))
add_data["is_post"] = add_data["post_comment_id"].apply(lambda text: text.split()[0] == "post")
                                                        
poor_data["post_comment_id_int"] = poor_data["post_comment_id"].apply(lambda text: int(text.split()[-1]))
poor_data["is_post"] = poor_data["post_comment_id"].apply(lambda text: text.split()[0] == "post")
                                                        
add_data.drop(["post_comment_id", "date", "name", "surname", "city", "region", "country"], axis=1, inplace=True)
poor_data.drop("post_comment_id", axis=1, inplace=True)

In [9]:
add_data

,author_id,wall_id,sex,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers,post_comment_id_int,is_post
0,85978354,112465206,2,1.0,0.0,0.0,0.0,0.0,0.0,,36.0,789.0,272.0,1,True
1,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,249,True
2,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,246,True
3,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,245,True
4,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,244,True
5,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,243,True
6,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,241,True
7,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,240,True
8,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,239,True
9,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,237,True


In [10]:
add_data.isnull().sum()

author_id                   0
wall_id                     0
sex                    114341
posts                  114341
comments               114341
likes                  114341
reposts                114341
res_postlikes          114341
res_comments           114341
university             114341
freinds                226633
groups                 226633
followers              226633
post_comment_id_int         0
is_post                     0
dtype: int64

In [11]:
add_data.shape

(8166504, 15)

In [12]:
add_data.drop_duplicates(subset=["wall_id", "post_comment_id_int", "author_id", "is_post"], inplace=True)

In [13]:
add_data.shape

(8079280, 15)

In [14]:
data = pd.merge(poor_data, add_data, on=["wall_id", "post_comment_id_int", "author_id", "is_post"], how="left")

In [15]:
data.head()

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
0,general lee,49154141,49154141,2012-02-24 11:54:20,Дмитрий,Филиппов,Владимир,Владимирская область,2,2404,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
1,shelby,49154141,49154141,2012-02-24 11:54:33,Дмитрий,Филиппов,Владимир,Владимирская область,1,2405,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
2,ford mustang,49154141,49154141,2012-02-25 22:27:35,Дмитрий,Филиппов,Владимир,Владимирская область,2,2411,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
3,murcielago,49154141,49154141,2012-02-24 11:53:51,Дмитрий,Филиппов,Владимир,Владимирская область,1,2400,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
4,суровый водитель,49154141,49154141,2012-02-23 16:22:56,Дмитрий,Филиппов,Владимир,Владимирская область,2,2392,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0


In [16]:
poor_data.shape, add_data.shape, data.shape

((5392586, 11), (8079280, 15), (5392586, 22))

In [17]:
data.isnull().sum()

text                        0
author_id                   0
wall_id                     0
date                        0
name                    48759
surname                 49346
city                   411426
region                 421852
text_length                 0
post_comment_id_int         0
is_post                     0
sex                    184556
posts                  184556
comments               184556
likes                  184556
reposts                184556
res_postlikes          184556
res_comments           184556
university             184556
freinds                263451
groups                 263451
followers              263451
dtype: int64

Ограничиваем данные постами, написанными позже 20 октября 2010 года и считаем их количество:

In [18]:
data_good_date = data[data.date >= datetime.date(year=2010,month=10,day=21)]
data_good_date.shape

(5121979, 22)

Определяем пользователей, у которых 50 и более слов за рассчётный период. Создаём с ними данные

In [19]:
groupedby_author_sum = data_good_date.groupby("author_id").agg({"text_length": "sum"})
groupedby_author_sum_50 = groupedby_author_sum[groupedby_author_sum["text_length"] >= 50]
authors_50 = groupedby_author_sum_50.index.values
data_authors_50 = data_good_date[data_good_date["author_id"].isin(authors_50)]

In [20]:
data_authors_50.shape

(4414265, 22)

Смотрим пользователей по регионам

In [21]:
data_authors_50["region"].value_counts()

Тюменская область                  202486
Пензенская область                 139186
Севастопольский городской совет    127923
Крым область                       100095
Дагестан                            98537
Орловская область                   91166
Архангельская область               90603
Коми                                90374
Ленинградская область               89177
Московская область                  87213
Краснодарский край                  83567
Мордовия                            78327
Санкт-Петербург                     75861
Москва                              74280
Ставропольский край                 72514
Вологодская область                 72444
Карелия                             72401
Псковская область                   71511
Нижегородская область               71079
Новосибирская область               70385
Северная Осетия - Алания            67505
Воронежская область                 67217
Татарстан                           64926
Хакасия                           

Возьмём пользователей с регионами, в которых более 10000 текстов

In [22]:
valid_regions = data_authors_50["region"].value_counts()[data_authors_50["region"].value_counts() > 10000].axes[0].tolist()
data_authors_50 = data_authors_50[data_authors_50["region"].isin(valid_regions)]
data_authors_50.isnull().sum()

text                        0
author_id                   0
wall_id                     0
date                        0
name                        0
surname                     0
city                        0
region                      0
text_length                 0
post_comment_id_int         0
is_post                     0
sex                     84463
posts                   84463
comments                84463
likes                   84463
reposts                 84463
res_postlikes           84463
res_comments            84463
university              84463
freinds                150747
groups                 150747
followers              150747
dtype: int64

In [23]:
data_authors_50.shape

(4163945, 22)

Авторов, у которых менее 300 слов, сливаем целиком за весь период.

In [26]:
groupedby_author_sum = data_authors_50.groupby("author_id").agg({"text_length": "sum"})
authors_300 = groupedby_author_sum[groupedby_author_sum["text_length"] <= 300].index.values
data_authors_300 = data_authors_50[data_authors_50["author_id"].isin(authors_300)]
data_authors_300.shape

(346635, 22)

In [27]:
aggregations = {"text": "sum", "region": "first", "city": "first", "sex": "first", "posts": "sum",
                "comments": "sum", "likes": "sum", "reposts": "sum", "res_postlikes": "sum",
                "freinds": "sum", "groups": "sum", "followers": "sum"}

In [28]:
merged_300 = data_authors_300.groupby("author_id").agg(aggregations)

In [29]:
merged_300

,res_postlikes,reposts,sex,comments,followers,text,posts,likes,freinds,city,groups,region
author_id,,,,,,,,,,,,
23835,0.0,0.0,1,25.0,33525.0,придти рыбка vk придти листик vk придти листик...,625.0,75.0,14200.0,Санкт-Петербург,1475.0,Санкт-Петербург
47882,6.0,0.0,2,0.0,684.0,машин ваз довольный почесть устраивать устраив...,21.0,6.0,360.0,Санкт-Петербург,114.0,Санкт-Петербург
78076,234.0,18.0,1,324.0,55170.0,прибывать удача экзамен юный кинуть подношение...,630.0,540.0,6696.0,Вологда,882.0,Вологодская область
106198,0.0,0.0,2,0.0,9656.0,викторий отправлять подарок энергия игра тайна...,64.0,0.0,5008.0,Санкт-Петербург,520.0,Санкт-Петербург
107733,260.0,160.0,2,120.0,80.0,помнить жизнь повторять огнь день победа vkont...,260.0,160.0,1600.0,Санкт-Петербург,240.0,Санкт-Петербург
133576,132.0,18.0,2,12.0,90.0,благодарность внимательный осторожный друг про...,630.0,6.0,1374.0,Санкт-Петербург,408.0,Санкт-Петербург
180542,56.0,14.0,1,7.0,1302.0,super vk видео саша сентябряhttp youtube watch...,70.0,49.0,3185.0,Санкт-Петербург,329.0,Санкт-Петербург
192888,0.0,0.0,1,0.0,571.0,добрый целить близкий девушка красавица форма ...,1.0,0.0,1006.0,Валдай,66.0,Новгородская область
209827,NaN,NaN,NaN,NaN,NaN,happy amy август ламбруско ждать поступать lov...,NaN,NaN,NaN,Москва,NaN,Москва


In [30]:
merged_300["text"] = merged_300["text"].apply(lambda x: [x])
merged_300.shape

(23877, 12)

merged_300 — слитые по каждому автору посты авторов, написавших от 50 до 300 слов включительно за рассчётный период.

Для авторов, у которых более 300 слов:
Берём каждого пользователя. Последовательно сливаем его тексты и проверям количество слов. Если в какой-то момент это количество достигло или превысило 300, формируем документ. Затем формируем следующий документ.
Если при такой нарезке последний кусок оказыается менее 50 слов, его к предыдущему.

In [31]:
groupedby_author_sum = data_authors_50.groupby("author_id").agg({"text_length": "sum"})
authors_more_300 = groupedby_author_sum[groupedby_author_sum["text_length"] > 300].index.values
data_authors_more_300 = data_authors_50[data_authors_50["author_id"].isin(authors_more_300)]
data_authors_more_300.shape

(3817310, 22)

Удаляем пользователей с id = 0.

In [32]:
data_authors_more_300 = data_authors_more_300[data_authors_more_300["author_id"] != 0]
data_authors_more_300.shape

(3616833, 22)

In [ ]:
from collections import defaultdict

# result = {author_id: {docs: [("text", likes), ...], probs: [(t_num, prob)...], region: "str", age: "str", sex: int}, ...}

merged_more_300 = {}
for author_id, author_data in data_authors_more_300.groupby("author_id"):
    texts_storage = ""
    texts_number = len(author_data["text"])
    
    auth_dict = {}
    auth_dict["docs"] = []
    auth_dict["region"] = author_data["region"].iloc[0]
    auth_dict["sex"] = author_data["sex"].iloc[0]
    auth_dict["city"] = author_data["city"].iloc[0]
    auth_dict["friends"] = author_data["friends"].iloc[0]
    auth_dict["groups"] = author_data["groups"].iloc[0]
    auth_dict["followers"] = author_data["followers"].iloc[0]
    auth_dict["region"] = author_data["region"].iloc[0]
    auth_dict["region"] = author_data["region"].iloc[0]
    
    
    for text_num, text in enumerate(author_data["text"]):
        last = (text_num + 1) == texts_number
        texts_storage += text
        
        if last:
            if len(texts_storage.split()) >= 50:
                # если это последний документ и количество слов в нём превышает 50 — формируем документ и формируем автора
                auth_dict["docs"].append(texts_storage)
                merged_more_300[author_id] = auth_dict
                texts_storage = "" 
            else:
                # а если не превышает — добавляем к предыдущему и формируем автора
                auth_dict["docs"][-1] += texts_storage
                merged_more_300[author_id] = auth_dict
                texts_storage = "" 
        else:
            if len(texts_storage.split()) >= 300:
                # если количество слов в документе достигло 300 — формируем документ
                auth_dict["docs"].append(texts_storage)
                texts_storage = ""
            else:
                # если ни одно условие не выполнено, продолжаем сливать
                continue

In [ ]:
merged_300.rename(columns={"text": "docs"}, inplace=True)
merged_300_dict = merged_300.to_dict(orient="index")

In [ ]:
len(merged_300_dict)

In [ ]:
general_dict = {**merged_more_300, **merged_300_dict}
len(general_dict)

In [ ]:
with open("/Users/Hun/hse_thesis/merged_texts_with_all_meta.pickle", "wb") as f:
    pickle.dump(general_dict, f)

In [ ]:
texts = []
for auth_id, val in general_dict.items():
    for text in val["docs"]:
        texts.append(text)

In [ ]:
dictionary = corpora.Dictionary([text.split() for text in texts])

In [ ]:
dictionary.save("/Users/Hun/hse_thesis/hse_thesis.dict_v3")

In [ ]:
corpus = [dictionary.doc2bow(text.split()) for text in texts]
corpora.MmCorpus.serialize("/Users/Hun/hse_thesis/hse_thesis.corpus_v3", corpus, metadata=True)

In [ ]:
lda150 = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=150)

In [ ]:
lda150.save("/Users/Hun/hse_thesis/lda150/hse_thesis_model150_v3.gen_mod")

In [ ]:
lda150.print_topics(num_topics=-1, num_words=10)

In [ ]:
lda150.minimum_probability = 0

In [ ]:
for user_id, val in general_dict.items():
    general_dict[user_id]["probs"] = []
    for doc in val["docs"]:
        probs = lda150[dictionary.doc2bow(doc.split())]
        general_dict[user_id]["probs"].append([(t_num, prob) for t_num, prob in probs])